In [1]:
###  RNA-seq data analysis in R ###
##  Основные шаги:
# 1. Загрузка таблицы каунтов и conditions
# 2. Аннотирование генов
# 3. Фильтрация от NA и низкоэкспрессированных генов
# 4.1 Нормализация (log2+quantile или vst/rlog) и визуализация (PCA plot, clustering, heatmap)
# 4.2 Нормализация в DESeq2 для выявления DEGs
# 5.1 limma
# 5.2 DESeq2
# 6. Анализ сигнальных путей на DEGs
# 7. GSEA pathway analysis
# 8. Отбор сигнальных путей по p.adj
# 9. GSEA plots visualization
# # 

# Установка библиотек:
# 1) install.packages("pkg_name")

# 2) Если библиотека из Bioconductor:
# if (!require("BiocManager", quietly = TRUE))
# install.packages("BiocManager")
# BiocManager::install("pkg_name")
#
# 3) Можно установить напрямую, скачав файл пакета
#
#Активация нужных библиотек (должны быть установлены заранее):

library(data.table)
library(DESeq2)
library(limma)
library(fgsea)
library(ggplot2)
library(ggrepel)
library("AnnotationDbi")
library(devtools)
library(dplyr)
library("org.Mm.eg.db")
#BiocManager::install("phantasus")
library(phantasus)
library(msigdbr)
library(clusterProfiler)
library(VennDiagram)
library(pheatmap)
library(utils)
library(EnhancedVolcano)
library(ggbeeswarm)

library(devtools)
library(rUtils)
library(ggrepel)
library(VennDiagram)
#BiocManager::install("pathview")
library(pathview)


# Загрузка таблицы экспрессии из базы данных GEO
# library(GEOquery)
# dir.create("cache")
# 
# ExpressionSet object:
# - Expression matrix – таблица каунтов – exprs(es)
# - fData – аннотация строк (название генов в разном формате)
# - pData – аннотация столбцов (conditions)
# 
# es <- getGEO("GSE53986", AnnotGPL = TRUE, destdir="cache")[[1]]
# str(experimentData(es))
# str(pData(es))       #таблица условий
# head(fData(es))
# fdata <-fData(es)
# fdata                #посмотрим на струкутру не с помощью head
# es$`treatment:ch1`   #образцы


# В папке с проектом открываем таблицу каунтов и условий к ней

#library(utils)
table <- read.table('./data/GSE150365_LMNA_raw_counts.txt', sep = '\t', header = T)
cond <- read.csv("./data/conditions.tsv", header=T, sep = '\t', row.names = 1)


# Удаляем букву "X" из названия образцов (колонки таблицы table). 
#Буква "X" осталась после подсчета каунтов.
tags <- sapply(colnames(table), function (x) sub("X", "", basename(x)))
colnames(table) <- tags

#Из всех условий выбраем миогенную дифференцировку (HS) и не дифференцированные клетки (control)
cond <- cond[which(cond$Treatment == "control" | cond$Treatment=="HS"),]

#Сделаем одинаковый порядок образцов в table (это название столбцов) и в cond 
#(названия строк).
#Это важно для запуска анализа дифф. экспрессии
table[,-1] <- table[,rownames(cond)]


# Посмотрим статистику

col_sums <- colSums(table[,-1])
col_sums #суммарное количество ридов на образец
plot(col_sums, xlab="sample") #простой график количества ридов
mean(col_sums) #среднее количество ридов во всем эксперименте
col_sums[col_sums < 7500000] #образцы, у которых меньше 7.5 миллиона ридов
boxplot(table[,c(2:9)]) #boxplot для выбранных образцов. Много генов с нулевым количеством каунтов.
#нужно делать фильтрацию

# Аннотирование строк
#table$Gene_id <- sub("\\..*", "", rownames(table)) #удаление .1 в ENSEMBL ID, если есть

table <- unique(table, by = "Gene_id_ENSEMBL") #оставляем гены с уникальным ensembl id 
#(если есть повторы, но не должно быть)
#library(data.table)
table <- as.data.table(table)
#Переводим ensembl в symbol
#library("AnnotationDbi")
#library("org.Mm.eg.db") #аннотация генов для мыши
table[, Symbol:= mapIds(org.Mm.eg.db, keys=Gene_id_ENSEMBL, 
                         keytype="ENSEMBL", column="SYMBOL")] #новый столбец symbol 
                                                              # по ENSEMBL id
#В столбце symbol есть NA значения. Нужно будет их потом убрать.

#Возможные типы клучей в базе данных
keytypes(org.Mm.eg.db)


# Можно сделать Expression Set object для экспорта в Phantasus
es <- ExpressionSet(as.matrix(table[,!c("Gene_id_ENSEMBL", "Symbol")])) #только каунты 
head(exprs(es),20)
fData(es) <- table[,c("Gene_id_ENSEMBL", "Symbol")] #название гена в различных базах данных
head(fData(es))
rownames(es) <- fData(es)$Gene_id_ENSEMBL #в качестве строк сделаем ensembl id
pData(es) <- cond
head(pData(es)) #таблица условий
# BiocManager::install("phantasus")
# library(phantasus)

write.gct(es,"es_raw.gct")


# Посмотрим на экспрессию гена бета-актина
exprs(es)[which(fData(es)$Symbol == "Actb"), ]
#Быстро сохраним график в формате png. Если нужно поменять оформление графика, 
#то лучше использовать пакет ggplot2
png("Actb.png", 1000, 800)
plot(exprs(es)[which(fData(es)$Symbol == "Actb"), ])
dev.off()


# Фильтрация. Удаляем NA и дибликаты в symbol
table <- table[!is.na(table$Symbol)]   # удаляем строки с NA в symbol
length(unique(table$Symbol))   # Проверяем есть ли дубликаты. 
#Есть, так как всего строк в table больше 32000, а уникальных из них 31950

# table <- unique(table, by="Symbol") # Можно удалить дубликаты так, 
                                       #но unique оставляет рандомно строки


# Дедубликация. Разным ensembl может соответствовать один symbol

# Для каждого ensembl id заменим экспрессию гена на медианное значение
#library(dplyr) #чтобы заработала функция %>% 
table_dedup <- table[,!c("Gene_id_ENSEMBL")] %>% group_by(Symbol) %>% 
                summarise_all(median) %>% as.data.table()  #выполняется долго

table_dedup[, Gene_id := table[match(table_dedup$Symbol, table$Symbol), "Gene_id_ENSEMBL"] ]  
# Добавили Gene_id ENSEMBL, она была удалена
table_dedup[, Entrez := mapIds(org.Mm.eg.db, keys=Symbol, keytype="SYMBOL", column="ENTREZID")] 
# Добавили Entrez id
table_dedup$Entrez <- as.character(table_dedup$Entrez) %>% replace(list = table_dedup$Entrez == "NULL", values = NA)  
# Заменили NULL на NA,если нет entrez 
table_dedup <- table_dedup[!is.na(table_dedup$Entrez)] # Remove Entrez NA


# ЦЕЛЬ: необходимо статистически сравнить средние экспрессий между двумя
# выборками образцов. Нулевая гипотеза говорит, что уровень экспрессии не будет различаться.
# 
# Возможные решения в классической статистике:
#   - т-тест (не подходит, так как не нормальное распределение данных)
#   - Манн-Уитни (слишком слабый, так как чаще всего мало образцов в группе)
# 
# Они нам не подходят.
# Для использования стат. критериев нужно оценить экспрессию и дисперсию для каждого гена.
# Для этого используют различные модели. 
# 
# Далее три пути анализа:
#   
# 1. Limma. На вход подаются нормализованные данные. 
# Использует линейные модели для моделирования экспрессии генов и их дисперсии. 
# Предполагается нормальность распределения данных для оценки коэффициентов модели. 
# Исходя из "одинаковости" коэффициентов модели перед факторами делают вывод об изменении экспрессии гена.
# 
# 2. DESeq2. На вход подаются "сырые" каунты, но после фильтрации. 
# Использует Generalized linear model (GLM) для моделирования экспрессии генов и их дисперсии.
# При оцнке коэффициентов модели используют отрицательное биномиальное распределение,
# которое более точно оченивает экспрессию. 
# 
# 3. EdgeR (не будем разбирать в рамках этого курса). 
# На вход подаются нормализованные данные. Тоже используется отрицательное биномиальное распределение.
# 
# 
# При анализе в модель включают потенциальные факторы, такие как бач-эффект или 
# экспериментальные условия, которые могут влиять на экспрессию генов. 
# Поэтому получается более точная оценка истинных дифференциальных паттернов экспрессии.
# 
# Достоверность определения коэффициентов линейной модели (p-value) в DESeq2 оценивается
# с помощью теста Вальда. 
# Если p-значение мало, мы отвергаем нулевую гипотезу и утверждаем, что ген дифференциально экспрессирован.


#Сделаем анализ дифференциальной экспрессии с помощью Limma

# # Профильтруем гены. Оставим только те, у которых среднее значение экспресии больше 1
table_dedup$mean <- rowMeans(table_dedup[,!c("Gene_id", "Symbol", "Entrez")])
table_dedup <- table_dedup[mean > 1,]
table_dedup$mean <- NULL # Удалим колонку "mean"

### Получили 15376 генов
# Сохраним
es <- ExpressionSet(as.matrix(table_dedup[,!c("Gene_id", "Symbol", "Entrez")]))
fData(es) <- table_dedup[, c("Gene_id", "Symbol", "Entrez")]
rownames(es) <- fData(es)$Gene_id
pData(es) <- cond
write.gct(es,"es_filter.gct")


# Принято использовать нормализацию log2+1, а потом quantile нормализацию
# Как было в phantasus
es.qnorm.top12K <- es       #скопировали объект es в новую переменную
#library(limma)
exprs(es.qnorm.top12K) <- normalizeBetweenArrays(log2(exprs(es.qnorm.top12K) + 1), method="quantile")  
# log2 and quantile normalization
#логарифмические отношения в среднем равнялись одному значению в каждом образце 

es.qnorm.top12K <- es.qnorm.top12K[head(order(apply(exprs(es.qnorm.top12K), 1, mean), 
                                              decreasing = TRUE), 12000), ]
#топ 12000 генов по среднему значению экспрессии

fData(es.qnorm.top12K)$mean <- apply(exprs(es.qnorm.top12K), 1, mean)  
# записываем среднее в аннотацию строк

write.gct(es.qnorm.top12K, file="./es.qnorm.top12k.gct")  #сохраним


## Анализ главных компонент (PCA plot)

# 1 способ
pca <- prcomp(t(exprs(es.qnorm.top12K)))
plot(pca$x[, 1:2])

# 2 способ
#install.packages("devtools")
#devtools::install_github("assaron/r-utils", force = T)
#library(rUtils)
#library(ggrepel)

p <- pcaPlot(es.qnorm.top12K, 1, 2) + 
  geom_text_repel(aes(label=samples)) +
  aes(color=Condition)
print(p)         # PCA plot colored for conditions
#другой способ сохранить график, если он сделан с помощью ggplot2
ggsave(p, width=12, height=8, file="PCA_1.png")

p <- pcaPlot(es.qnorm.top12K, 1, 2) + 
  geom_text_repel(aes(label=samples)) +
  aes(color=Day)+
  stat_ellipse(aes(color=Day))
print(p)           # Colored by Day of differentiation
ggsave(p, width=12, height=8, file="PCA_2.png")



# Кластеризация: hierarchical and k-means
#library(pheatmap)
# z-score нормализация. Среднее значение всех значений равно 0, а стандартное отклонение равно 1.
es.qnorm.top12K.z <- es.qnorm.top12K
exprs(es.qnorm.top12K.z) <- t(apply(exprs(es.qnorm.top12K.z),1,scale)) # транспонировали
phmap_z <- pheatmap(exprs(es.qnorm.top12K.z), 
                    kmeans_k=5, #количество кластеров определяется методом "логтя" или "колена"
                    clustering_distance_cols = "correlation", 
                    clustering_method = "average",
                    annotation_col = cond,
                    color=colorRampPalette(c("navy", "white", "red"))(50))
head(phmap_z$kmeans$cluster, 20)  # какие гены к каким кластерам принадлежат

#Здесь выбрали количество кластеров сами, но более правильно использовать метод "логтя" или "колена"


## Удалим выбросы среди образцов (выявили с помощью phantasus):
outliers <- c("wt_HS_term_1","482_HS_2d_1")
es.qnorm.top12K.no_out <- es.qnorm.top12K[, !(colnames(es.qnorm.top12K) %in% outliers)]
colnames(es.qnorm.top12K.no_out)



## Удалим их из сырых данных и обновим ExpressionSet
# Удалим образцы из "table_dedup" (unnormalized table!)
table_dedup <- as.data.frame(table_dedup)
table_dedup <- table_dedup[,!(names(table_dedup) %in% outliers)] %>% as.data.table()
cond <- cond[!(rownames(cond) %in% outliers),]

#Новый Expression Set
es <- ExpressionSet(as.matrix(table_dedup[, !c("Gene_id", "Symbol", "Entrez")]))
fData(es) <- table_dedup[, c("Gene_id", "Symbol", "Entrez")]
rownames(es) <- fData(es)$Gene_id
pData(es) <- cond
write.gct(es, "es_filt_no_out.gct")





## Differential Expression Analysis (DEA) использую Limma
#Анализ дифференциальной экспрессии
#выбираем, какой столбец будет спользован для разбиения на группы сравнения
#В основе - линейная модель (линейная регрессия), где мы хотим предсказать уровень экспрессии каждого гена. 
#В зависимости от условий.
#
es.design <- model.matrix(~0+Condition, data=pData(es.qnorm.top12K.no_out)) 
es.design
#library(limma)
fit <- lmFit(es.qnorm.top12K.no_out, es.design)

fit1 <- contrasts.fit(fit, makeContrasts2(c("Condition", "232_control", "wt_control"),
                                          levels=es.design)) #парное сравнение
fit1 <- eBayes(fit1)
de  <- topTable(fit1, adjust.method="BH", number=Inf) #поправка на множественные сравнения
head(de)
de <- as.data.table(de, keep.rownames=TRUE)
de  <- de[order(t, decreasing = T)]
de$logFC <- as.numeric(de$logFC)
de_filt <- de[adj.P.Val<0.05 & (logFC > 1.5 | logFC < -1.5)] 
# Отобрали дифф. экспрессирующиеся гены (DEGs)

#Построим Volcanoplot
volcanoplot(fit1, style = "p-value", highlight = 116, names = fit1$genes$Symbol,
            xlab = "Log2 Fold Change", ylab = NULL, pch=16, cex=0.35)

# Более красивый график
#library(EnhancedVolcano)
volcano_limma <- EnhancedVolcano(de, title = "Volcanoplot_Limma",
                lab = fit1$genes$Symbol, #ylim = c(0,10),
                x = 'logFC',
                y = 'adj.P.Val', pCutoff = 0.05,
                FCcutoff = 1.5,)
ggsave(volcano_limma, width=10, height=10, file="Volcanoplot_Limma.png")

# Другое парное сравнение "482_control" vs "wt_control" 

fit2      <- contrasts.fit(fit, makeContrasts2(c("Condition", "482_control", "wt_control"),
                                               levels=es.design))
fit2      <- eBayes(fit2)
de2        <- topTable(fit2, adjust.method="BH", number=Inf)
head(de2)
de2        <- as.data.table(de2, keep.rownames=TRUE)
de2        <- de2[order(t, decreasing = T)]
de2_filt   <- de2[adj.P.Val<0.05 & (logFC > 1.5 | logFC < -1.5)] #DEGs

# Пересечение upregulated DEGs двух сравненний с помощью диаграмм Венна
#library(VennDiagram)
set1 <- de_filt[logFC > 1.5, Symbol] 
set2 <- de2_filt[logFC > 1.5, Symbol] 
venn.diagram(
  x = list(set1, set2),
  category.names = c("232_vs_wt" , "482_vs_wt"),
  filename = 'venn_diagramm.png',
  output=TRUE, 
  height = 1000, 
  width = 1000, 
  resolution =300, 
  fill = c("#E69F00", "#56B4E9"),
  lty = 'blank',
  cat.cex = 0.6,
  cat.fontface = "bold",
  cat.default.pos = "outer",
  cat.pos = c(0, -27),
  cat.dist = c(0.01, 0.02),
  cat.fontfamily = "sans")
#сохранили в файл venn_diagramm.png

intersect(set1, set2) # Пересечение upregulated DEGs
setdiff(set2, set1) # Уникальные upregulated DEGs для 482_vs_WT


# pathway analysis with fgsea
stats <- de[, setNames(t, Entrez)] 
#выбрали из таблицы de только столбцы t и названия генов в столбце Entrez

#library(msigdbr)
# GO BP pathways from MSigDB
m_df <- msigdbr(species = "Mus musculus", category = "C5", subcategory = "BP")
m_df
pathways <- split(m_df$entrez_gene, m_df$gs_name) #entrez id и название пути
#подгрузили базу к себе 

#library(fgsea)
#пересекаем наши гены с генами из базы
frML <- fgseaMultilevel(pathways, stats,
                          sampleSize = 100,
                          nproc=4, minSize=15, maxSize=500)
frML <- frML[order(padj)] 
frML <- frML[padj < 0.05]

collapsed_pathways <- collapsePathways(frML, pathways, stats)
#схлопываем похожие сигнальные пути
str(collapsed_pathways) 

main_pathways <- frML[pathway %in% collapsed_pathways$mainPathways][
  order(sign(ES)*log(pval)), pathway] # отбираем только main pathways по значению ES и p-value

frMain <- frML[match(main_pathways, pathway)] #оставляем только main пути
frMain[, leadingEdge := lapply(leadingEdge, mapIds, 
                                 x=org.Mm.eg.db, keytype="ENTREZID", column="SYMBOL")] 
#переименуем гены из Entrez id в Symbol
#добавили столбец symbol, содержащий название генов, участвующих в данном сигнальном пути

dir.create("gsea") #сделали папку
fwrite(frMain, file="gsea/FGSEA_filtered.tsv", sep="\t", sep2=c("", " ", "")) 
# сохранили в виде tsv файла. Можно в другом формате.


# Построим GSEA plot для определенного сигнального пути
gsea_plot <- plotEnrichment(pathways[["GOBP_DNA_REPLICATION"]], stats) + 
  ggtitle("GOBP_DNA_REPLICATION")
ggsave(gsea_plot, width=8, height=4, file="gsea/GOBP_DNA_REPLICATION.png")













## Поиск дифференциально экспрессированных генов с помощью DESeq2
## На вход подается фильтрованная и не нормированная таблица каунтов
## Возьмем для анализа 15000 генов (можно все)


library(DESeq2)
exprs(es) <- apply(exprs(es), 2, as.integer) #Взяли уже имеющийся датасет es, сделали тип integer

# Внутри DESeq2 своя нормализация, основанная на медианной экспрессии. Сейчас ее сделаем.
dds <- DESeqDataSetFromMatrix(countData = exprs(es), 
                              colData = pData(es),
                              design=~Condition)     # Condition это наши группы сравнения
dds <- dds[rowSums(counts(dds)) > 10, ] #отфильтруем гены, у которых в сумме по всем образцам меньше 10 каунтов
dds$Condition <- relevel(dds$Condition, ref = "wt_control") #указали группу с "базовым" уровнем экспрессии
dds <- DESeq(dds) #сам DESeq
dds


# Построим PCA plot
#Для начала нужно сделать vst нормализацию данных
vst <- varianceStabilizingTransformation(dds) # эта функция работает на основе dds

plotPCA(vst,intgroup=c("Condition")) + 
  geom_text_repel(aes(label=name))+
  coord_fixed(ratio = 4)

plotPCA(vst,intgroup=c("Day")) + coord_fixed(ratio = 4)+
  geom_text_repel(aes(label=name))+
  stat_ellipse(aes(color=Day))

plotPCA(vst,intgroup=c("Cell_type")) + 
  coord_fixed(ratio = 4)+
  geom_text_repel(aes(label=name))

ggsave("PCA_plot_DESeq2.png") #если хотите сохранить график

#Иерархическая кластеризация образцов
dists <-  dist(t(assay(vst)))
plot(hclust(dists)) # иерархическая кластеризация образцов
#трипликаты вместе

# Построим тепловую карту 20 самых вариабельных генов
deseq_counts <- assay(vst) 

deseq_counts.z <- t(apply(deseq_counts,1,scale)) # scaling
colnames(deseq_counts.z) <- colnames(deseq_counts)
deseq_counts.z <- deseq_counts.z[c(1:20),] #топ-20 генов (для удобства и быстроты)
#можно здесь выбрать интересующие вас гены
pheatmap(deseq_counts.z, 
         #kmeans_k=15,                      
         clustering_distance_cols = "correlation", 
         clustering_method = "average",
         annotation_col = cond,
         color=colorRampPalette(c("navy", "white", "red"))(50)) #50 шаг цветовой градации
#можно сделать k-means кластеризацию


# Найдем дифференциально экспрессирующиеся гены 232_control vs wt_control
dir.create("./deseq2/", showWarnings = F)  #output directory
unique(dds$Condition)
# Важен порядок написания образцов в contrasts
#На первом месте указываем "воздействие", а на стором то, с чем сравниваем, т.е. контроль
# log2FC = 232_control -  wt_control   
# If  log2FC>0 => gene is upregulated in 232_control; if log2FC<0 => gene is downregulated

# Уточним группы сравнения
deseq2_res <- results(dds, contrast = c("Condition", "232_control", "wt_control"), cooksCutoff = F)
head(deseq2_res)
deseq2_res <- data.table(ID=rownames(deseq2_res), as.data.table(deseq2_res))  
head(deseq2_res)
tail(deseq2_res)


head(fData(es)) #разные аннотации генов (ensembl, symbol, entrez)
deseq2_res <- cbind(deseq2_res, fData(es)) # добавили аннотацию таблице дифф. генов
deseq2_res <- deseq2_res[order(stat, decreasing = T), ] #сортировка по переменной stat
#stat - статистика Вальда
head(deseq2_res)

# Myh7 ген
deseq2_res[Symbol == "Myh7"]

#library(ggbeeswarm)
geneCounts <- plotCounts(dds, gene = "ENSMUSG00000053093", intgroup=c("Condition"),returnData = TRUE)
ggplot(geneCounts, aes(x = Condition, y = count, colour = Condition, size = 3)) +
  scale_y_log10() +  geom_beeswarm(cex = 3) #+ geom_point(size=3)

# Сохраним табличку
fwrite(deseq2_res, file="./deseq2/232_control_vs_wt_control.de.tsv", sep="\t")

# Volcanoplot

#library(EnhancedVolcano)
#png("Volcanoplot_232_contr_vs_wt_contr.png", 800, 800, pointsize=30)
Volcano <- EnhancedVolcano(deseq2_res, title = "Volcanoplot_DESeq2",
                           lab = deseq2_res$Symbol, #ylim = c(0,10),
                           x = 'log2FoldChange',
                           y = 'padj', pCutoff = 0.05,
                           FCcutoff = 1.5,)
Volcano
#dev.off()


## Pathway analysis

## Сделаем анализ сигнальных путей на DEGs (Differentially Expressed Genes)
## Будем использовать базу данных GO BP (Gene Ontology - Biological Processes)
# UP and DOWN DEGs анализируются отдельно
#library(clusterProfiler)

deg_filt <- deseq2_res[padj<0.05 & (log2FoldChange > 1.5 | log2FoldChange < -1.5)]

genes <- deg_filt[log2FoldChange > 1.5]$Symbol

go_deg <- enrichGO(genes, 'org.Mm.eg.db', ont="BP", pvalueCutoff=0.05, keyType = "SYMBOL")
head(go_deg)
head(go_deg$Description)

barplot(go_deg, showCategory=10) # Первые 10 путей с p.adj < 0.05 
dotplot(go_deg, showCategory=10)

go_deg_res <- go_deg@result
go_deg_res <- go_deg_res %>% dplyr::filter(go_deg_res$p.adjust < 0.05)
fwrite(go_deg_res,"EnrichGO_Up.csv")


# Можно использовать другие базы данных сигнальных путй (как в R, так и онлайн).
# Из онлайн тулов можно использовать Msigdb и Enrichr

# Теперь попоробуем KEGG базу данных для анализа сигнальных путей

genes <- deg_filt[log2FoldChange > 1.5]$Entrez  #FC>1.5, p.adj=0.05
kegg_deg <- enrichKEGG(genes, organism = "mmu", keyType = 'ncbi-geneid', pvalueCutoff = 0.05)
barplot(kegg_deg, showCategory=10) # Первые 10 путей с p.adj < 0.05  

#Визуализируем сигнальный путь Calcium signaling pathway
#BiocManager::install("pathview")
#library(pathview)
de_genes <- deseq2_res[abs(log2FoldChange) > 1.5 & padj < 0.05,]
genes <- de_genes$log2FoldChange
names(genes) <- de_genes$Entrez
dme <- pathview(gene.data=genes, pathway.id="mmu04020", species = "mmu")
# Открываем сохраненные .png файлы




## GSEA - Gene Set Enrichment Analysis

#library(clusterProfiler)
genes <- as.vector(deseq2_res$stat) #без фильтрации по p.adj и LFC
names(genes) <- as.vector(deseq2_res$Symbol)
genes <- sort(genes, decreasing = TRUE) #именованный список генов, отсортированных по stat
gse <- gseGO(geneList     = genes,
             OrgDb        = org.Mm.eg.db,
             ont          = "BP",
             keyType = "SYMBOL",
             nPerm=10000)

# Dot plot of GSEA
dotplot(gse, showCategory=10, split=".sign") + facet_grid(.~.sign) #+ ggtitle("dotplot for GSEA")
gseaplot(gse, geneSetID = 5, by = "runningScore", title = gse$Description[5]) #визуализируем 5-ый сигнальный путь


# Теперь используем fgsea package
# Скачаем пути из базы данных

#library(msigdbr)
# Gene Ontology Biological Processes pathways from MSigDB (https://www.gsea-msigdb.org/gsea/msigdb/index.jsp)
m_df <- msigdbr(species = "Mus musculus", category = "C5", subcategory = "BP")
m_df
pathways <- split(m_df$entrez_gene, m_df$gs_name)

# t statistics
stats <- deseq2_res[, setNames(stat, Entrez)]

#library(fgsea)
fr <- fgsea(pathways, stats, nperm = 100000, nproc=4, minSize=15, maxSize=500)
fr[order(padj)]
fr_res <- fr[order(NES)][padj < 0.05]

#сделаем новый столбец, в котором будут указаны символы генов, участвующих в сигнальном пути
for (val in 1:nrow(fr_res)){
  fr_res$symbol[[val]] <- fr_res$leadingEdge[[val]] %>% mapIds(org.Mm.eg.db, keys = ., keytype = "ENTREZID",column = c('SYMBOL'), multiVals="first") %>%  paste(collapse = ", ")
  val = val+1
} #выполняется долго

# Сохраним
dir.create("gsea")
fwrite(fr_res, file="gsea/fgsea_232_contr_vs_wt_contr.csv", sep=",")

# Построим GSEA plots
topPathwaysUp <- fr_res[ES > 0 & padj < 0.05][head(order(padj), n=10), pathway]
topPathwaysDown <- fr_res[ES < 0 & padj < 0.05][head(order(padj), n=10), pathway]
topPathways <- c(topPathwaysUp, rev(topPathwaysDown))

FGSEA <- plotGseaTable(pathways[topPathways], stats, fr_res, 
                       gseaParam=0.5)

FGSEA
ggsave(file="Gsea_plot_232_c_vs_wt_c.jpeg", dpi=300, width = 14, height = 4)


# GSEA plots 

plotEnrichment(pathways[["GOBP_MUSCLE_CONTRACTION"]],
               stats, ticksSize=0.5) + 
  labs(title="Muscle contraction")+
  theme(plot.title = element_text(size=30, color="black"),
        axis.text = element_text(color = "black", size = 18), 
        axis.title.y = element_text(size=25, color="black"),
        axis.title.x = element_text(size=25, color="black")
  )
ggsave(file="Muscle_contraction.jpeg", dpi=300, width = 8, height = 4)


plotEnrichment(pathways[["GOBP_MITOCHONDRIAL_RESPIRATORY_CHAIN_COMPLEX_ASSEMBLY"]],
               stats, ticksSize=0.5) + 
  labs(title="MITOCHONDRIAL RESPIRATORY CHAIN COMPLEX ASSEMBLY")+
  theme(plot.title = element_text(size=15, color="black"),
        axis.text = element_text(color = "black", size = 18), 
        axis.title.y = element_text(size=25, color="black"),
        axis.title.x = element_text(size=25, color="black")
  )
ggsave(file="Mitochondrial_respiration.jpeg", dpi=300, width = 8, height = 4)











In [2]:
BiocManager::install(c("DESeq2", "limma", "fgsea", "AnnotationDbi", "org.Mm.eg.db", "phantasus", "clusterProfiler", "pathview"))

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.16 (BiocManager 1.30.22), R 4.2.3 (2023-03-15)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'DESeq2' 'AnnotationDbi'”
Installing package(s) 'limma', 'fgsea', 'org.Mm.eg.db', 'phantasus',
  'clusterProfiler', 'pathview'

also installing the dependencies ‘prettyunits’, ‘progress’, ‘gridGraphics’, ‘tweenr’, ‘polyclip’, ‘RcppEigen’, ‘gridExtra’, ‘lazyeval’, ‘clipr’, ‘hms’, ‘vroom’, ‘tzdb’, ‘R.oo’, ‘R.methodsS3’, ‘highr’, ‘xfun’, ‘yaml’, ‘mvtnorm’, ‘DEoptimR’, ‘ggfun’, ‘ggplotify’, ‘patchwork’, ‘ggforce’, ‘viridis’, ‘tidygraph’, ‘graphlayouts’, ‘cpp11’, ‘ape’, ‘tidytree’, ‘treeio’, ‘readr’, ‘xml2’, ‘R.utils’, ‘brew’, ‘later’, ‘promises’, ‘knitr’, ‘remotes’, ‘webutils’, ‘rappdirs’, ‘zip’, ‘Rhdf5

In [3]:
update.packages(ask = FALSE)

Warning message in install.packages(update[instlib == l, "Package"], l, contriburl = contriburl, :
“installation of package ‘bit’ had non-zero exit status”
Warning message in install.packages(update[instlib == l, "Package"], l, contriburl = contriburl, :
“installation of package ‘bitops’ had non-zero exit status”
Warning message in install.packages(update[instlib == l, "Package"], l, contriburl = contriburl, :
“installation of package ‘cli’ had non-zero exit status”
Warning message in install.packages(update[instlib == l, "Package"], l, contriburl = contriburl, :
“installation of package ‘colorspace’ had non-zero exit status”
Warning message in install.packages(update[instlib == l, "Package"], l, contriburl = contriburl, :
“installation of package ‘curl’ had non-zero exit status”
Warning message in install.packages(update[instlib == l, "Package"], l, contriburl = contriburl, :
“installation of package ‘digest’ had non-zero exit status”
Warning message in install.packages(update[instlib